<a href="https://colab.research.google.com/github/nick1287/Airlines/blob/main/Categorizacion_Modelo_Culpabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


BERT en un problema de clasificación
====================================

# Base de datos SQL

---Armo la descripción del siniestro
drop table  #aux_descripcion
select distinct ejercicio, siniestro, sinister_detail
into #aux_descripcion
from CLAIMS.[GW].[CLAIM_BI_ExportADIC_OK]
--where Ejercicio >=2022


select * from CLAIMS.[GW].[CLAIM_BI_ExportADIC_OK]
 where Siniestro = 1348579

select * from #aux_descripcion
--(187614 rows affected)

---Nos quedamos con la máxima fecha de actualización del siniestro

select siniestro, item, subsiniestro, max(fecha) as fecha
into #aux_max_fecha
from claims.[GW].[CLAIM_BI_Stock_Pasivo_ok]
group by siniestro, item, subsiniestro

--(266961 rows affected)

---Nos quedamos con la categoría más actual

select distinct a.siniestro, a.item, a.subsiniestro, a.fecha, b.categoria
into #aux_categoria
from #aux_max_fecha a
inner join claims.[GW].[CLAIM_BI_Stock_Pasivo_ok] b
on a.siniestro = b.siniestro and a.item = b.item and a.subsiniestro = b.subsiniestro and a.fecha = b.fecha

select distinct categoria from claims.[GW].[CLAIM_BI_Stock_Pasivo_ok]

select * from claims.[GW].[CLAIM_BI_Stock_Pasivo_ok] where categoria = 'activo'

select * from #aux_categoria
where Siniestro = 1212553

select * from procesosclaims.dbo.entrenamiento_culpable
where num_Stro = 1212553
--(266990 rows affected)

---Me quedo con los datos del siniestro de simba_stro de RCD del 2019 en adelante

/*
select
a.num_pol,
a.num_ite,
a.num_end,
a.cod_stro_di,
Stro = convert(int, substring(cod_stro_di,7,7)),
NSST = convert(int, substring(cod_stro_di,17,3)),
Nite =convert(int, substring(cod_stro_di,14,3)),
fec_proc_min=min(a.fec_proc),
fec_proc_max=max(a.fec_proc)
into #aux_siniestros
from MONITORING_DW.dbo.simba_stro a
left join Claims.dbo.dim_reclamo d
on a.cod_reclamo = d.cod_reclamo
where a.cod_sec = 4 and a.cod_ajuste = 1
and fec_ocu > 20230400 and d.modelo in ('RCD')
group by
a.num_pol,
a.num_ite,
a.num_end,
a.cod_stro_di,
convert(int, substring(a.cod_stro_di,7,7)),
convert(int, substring(cod_stro_di,17,3)),
convert(int, substring(cod_stro_di,14,3))
*/
--(162791 rows affected)

drop table #aux_siniestros
select
a.num_pol
,a.num_ite
--a.num_end,
--a.cod_stro_di,
,a.num_Stro--Stro = convert(int, substring(cod_stro_di,7,7)),
,a.num_sstro--NSST = convert(int, substring(cod_stro_di,17,3)),
--,Nite =convert(int, substring(cod_stro_di,14,3)),
--fec_proc_min=min(a.fec_proc),
--fec_proc_max=max(a.fec_proc)
into #aux_siniestros
from claims.dbo.fact_siniestros_ingresados a--MONITORING_DW.dbo.simba_stro a
left join claims.BasesRsa.codrecl B--Claims.dbo.dim_reclamo d
on a.cod_reclamo = B.codrecl
left join reporting_claims.dbo.productores c
on a.cod_pro = c.cod_pro
where a.num_sec = 4 --and a.cod_ajuste = 1
--and fec_ocu > 20230400 and d.modelo in ('RCD')
and b.agrup3 = 'RCD'
and canal not like '%Uber%'
group by
a.num_pol,
a.num_ite
--a.num_end
--a.cod_stro_di,
--convert(int, substring(a.cod_stro_di,7,7)),
--convert(int, substring(cod_stro_di,17,3)),
--convert(int, substring(cod_stro_di,14,3))
,a.num_Stro--Stro = convert(int, substring(cod_stro_di,7,7)),
,a.num_sstro--NSST = convert(int, substring(cod_stro_di,17,3)),
--,a.num_ite --Nite =convert(int, substring(cod_stro_di,14,3)),

--drop table #aux_base

drop table #ver
select distinct  a.*, b.sinister_detail, c.categoria
into #ver--claims.dbo.culpable_o_no_20230310
from #aux_siniestros a
inner join #aux_descripcion b
on a.num_stro=b.siniestro
inner join #aux_categoria c
on a.num_stro=c.siniestro and a.num_sstro=c.Subsiniestro
and a.num_ite=c.Item
where (categoria in ('transable') or left(categoria,10)='defendible')---('Sin clasificar')
and len (SINISTER_DETAIL) > 20
union all
select distinct  a.*, b.sinister_detail, c.categoria
from #aux_siniestros a
inner join #aux_descripcion b
on a.num_stro=b.siniestro
inner join #aux_categoria c
on a.num_stro=c.siniestro and a.num_sstro=c.Subsiniestro
and a.num_ite=c.Item
where len(sinister_detail) <20 and (categoria in ('transable') or left(categoria,10)='defendible')---('Sin clasificar')
 and sinister_detail like '%me choco%'
union all
select distinct  a.*, b.sinister_detail, categoria = 'No_aplica'--c.categoria
from #aux_siniestros a
inner join #aux_descripcion b
on a.num_stro=b.siniestro
inner join #aux_categoria c
on a.num_stro=c.siniestro and a.num_sstro=c.Subsiniestro
and a.num_ite=c.Item
where (categoria in ('transable') or left(categoria,10)='defendible')---('Sin clasificar')
and len (SINISTER_DETAIL) < 20 and sinister_detail not like '%me choco%'

--68358

select distinct CATEGORIA from #ver

select distinct SINISTER_DETAIL from #ver where len (SINISTER_DETAIL) < 20 --like  'sin denuncia'

select COUNT (*) from #ver where len (SINISTER_DETAIL) <5 --like  'sin denuncia'

select * from  #ver where SINISTER_DETAIL = 'h'

update a
set categoria = 'No_aplica'
from #ver a
where SINISTER_DETAIL like  '%sin denuncia%' or SINISTER_DETAIL like '%s/ denuncia%'

select * from #ver
/*
drop table #sacar
select *
into #sacar
from #ver
where SINISTER_DETAIL like  '%sin denuncia%' or SINISTER_DETAIL like '%s/ denuncia%'

drop table procesosclaims.dbo.entrenamiento_culpable
select a.*
into procesosclaims.dbo.entrenamiento_culpable
from #ver a
left join #sacar b
on a.num_stro = b.num_stro
and a.num_sstro = b.num_sstro
and a.num_ite = b.num_ite
where b.num_stro is null

update a
set categoria = 'Defendible'
from procesosclaims.dbo.entrenamiento_culpable a
where CATEGORIA like 'Defendible%'
*/

update a
set categoria = 'No_Culpable'
from #ver a--procesosclaims.dbo.entrenamiento_culpable a
where CATEGORIA like 'Defendible%'

update a
set categoria = 'Culpable'
from #ver a--procesosclaims.dbo.entrenamiento_culpable a
where CATEGORIA like 'transable%'

drop table procesosclaims.dbo.entrenamiento_culpable
select *
into procesosclaims.dbo.entrenamiento_culpable
from #ver

select concat (num_pol,';'), concat (num_ite,';'), concat (num_stro,';'), concat(num_sstro,';'),concat (sinister_detail,';')
--sinister_detail=replace(replace(replace(replace(sinister_detail, char(9), ''),char(10), ''),char(13), ''),char(4), '')
,concat (categoria,';')


select num_pol,num_ite,num_stro,num_sstro,sinister_detail
--sinister_detail=replace(replace(replace(replace(sinister_detail, char(9), ''),char(10), ''),char(13), ''),char(4), '')
,categoria
from procesosclaims.dbo.entrenamiento_culpable
group by num_pol,num_ite,num_stro,num_sstro,sinister_detail
--sinister_detail=replace(replace(replace(replace(sinister_detail, char(9), ''),char(10), ''),char(13), ''),char(4), '')
,categoria

select * from claims.dbo.fact_siniestros_ingresados where num_sec = 4 order by fecha_carga desc

select * from Claims.dbo.fact_siniestros_ingresados where num_sec = 4 and num_pol = 50032782

select * from MONITORING_DW.dbo.simba_Di where num_pol = 50032782

select * from claims.dbo.fact_siniestros_reservas
where aaaamm_rva = 202306 and num_sec in (2,22) and reserva >1000000

select *
from procesosclaims.dbo.entrenamiento_culpable

sp_help'entrenamiento_culpable'

select *
from procesosclaims.dbo.entrenamiento_culpable
where num_pol = 5916146



(72485 rows affected)

select * from procesosclaims.dbo.entrenamiento_culpable
drop table claims.dbo.culpable
select distinct  a.*, b.sinister_detail, c.categoria
into claims.dbo.culpable
from #aux_siniestros a
inner join #aux_descripcion b
on a.num_stro=b.siniestro
inner join #aux_categoria c
on a.num_stro=c.siniestro and a.num_sstro=c.Subsiniestro
and a.num_ite=c.Item
where CATEGORIA like 'sin clasificar' and fecha >= '2023-01-01'


select * from claims.dbo.culpable

select * from #ver where len(SINISTER_DETAIL) < 5

select * from #entrenamiento where len(SINISTER_DETAIL) < 30

Select * from #entrenamiento where SINISTER_DETAIL like 'SIN DENUNCIA - MED'


----(66443 rows affected)

select *
from pricing.dbo.culpable_o_no_20230310

select categoria, count(*)
from #aux_base
group by categoria
order by 1




select count(*) from #aux_base
where sinister_detail is null

select count(*) from #aux_base
where categoria is null


--103176


select iif(res_brt_rsa=0,'no culpable','culpable'), count(*)
from #aux_base
where sinister_detail is not null
group by iif(res_brt_rsa=0,'no culpable','culpable')

--(No column name)	(No column name)
--culpable	48597
--no culpable	54579

select *,estado=iif(res_brt_rsa=0,'no culpable','culpable')
from #aux_base
where sinister_detail is not null

select distinct a.*, b.fecha as fecha_ult_cat, b.categoria, d.canal, d.subcanal, e.Ejercicio, e.DESC_TRAMITADOR
,cod_estado = iif (f.cod_estado is null, 90, f.cod_estado)
from ProcesosClaims.dbo.prediccion_20230516 a
left join #aux_categoria b
on a.num_Stro = siniestro and a.num_sstro = b.subsiniestro and a.num_ite = b.item
left join (select * from Claims.dbo.fact_siniestros_ingresados) c
on a.num_Stro = c.num_stro and a.num_sstro = c.num_sstro and a.num_ite = c.num_ite
left join Reporting_Claims.dbo.productores d
on c.cod_pro = d.cod_pro
left join claims.[GW].[CLAIM_BI_ExportADICSubs_OK] e
on a.num_Stro = e.Siniestro and a.num_sstro = e.Subsiniestro and a.num_ite = e.Item
left join (select * from Claims.dbo.fact_siniestros_reservas where aaaamm_rva = 202306 and num_sec = 4) f
on a.num_Stro = f.num_stro and a.num_sstro = f.num_sstro and a.num_ite = f.num_ite

select * from #aux_categoria where siniestro = 1302973

# Nueva sección

Introducción
------------

Los modelos basados en transformers nos pueden ayudar a resolver varios tipos de problemas. Desde problemas de clasificación y regresión hasta tareas más complejas como resumen de textos o generación de leguaje condicionado. Veamos como resolver el problema de clasificación de tweets sobre el que hemos estado trabajando anteriormente pero ahora utilizando el modelo BERT.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/BERT.txt \
    --quiet --no-clobber
!pip install -r BERT.txt --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Cargamos el set de datos

In [ ]:
#Librería para conectarse a SQL

#importar pandas
import pandas as pd
base = pd.read_excel("entrenamiento_Prueba_2.xlsx")

In [ ]:
base

,num_pol,num_Stro,sinister_detail,categoria,Caracteres
0,5380929,1156984,.,No_aplica,1
1,999999999,1340973,s,No_aplica,1
2,5420691,1146105,-,No_aplica,1
3,7050199,1268001,H,No_aplica,1
4,5538353,1147194,De,No_aplica,2
...,...,...,...,...,...
51427,7146656,1293411,El día 30/04 estando estacionado mi Peugeot 40...,No_Culpable,500
51428,7372593,1361076,Yo estaba parado en el semáforo aguardando la ...,No_Culpable,500
51429,50021011,1267063,Venía por Av. Alberdi por mi mano cuando de re...,No_Culpable,500
51430,7221615,1320446,"Se contacta empleado del asegurado, denuncia q...",Culpable,500


In [ ]:
base=base.dropna()

In [ ]:
#X_train=base["sinister_detail"][(base["randomnum"]!=9) & (base["randomnum"]!=10)]
#X_test=base["sinister_detail"][(base["randomnum"]==9) | (base["randomnum"]==10)]
#y_train=base["estado"][(base["randomnum"]!=9) & (base["randomnum"]!=10)]
#y_test=base["estado"][(base["randomnum"]==9) | (base["randomnum"]==10)]

In [ ]:
#X_train.head()

In [ ]:
#X_test.head()

In [ ]:
#y_train.head()

In [ ]:
#y_test.head()

In [ ]:
#Separamos training y testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(base["sinister_detail"], base["categoria"],
                                                    test_size=0.2,
                                                    stratify=base["categoria"])

### Verificando el hardware disponible

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Este notebook se está ejecutando en", device)

Este notebook se está ejecutando en cuda


Transferencia de Aprendizaje y Fine-tuning
------------------------------------------

En general, existen 2 estrategias para utilizar modelos de lenguaje pre-entrenados en una tarea especifica:
 - Feature-based
 - Fine-tunning

Las técnicas que se conocen como **Feature-based** utiliza arquitecturas especificas para resolver cada una de las tareas de NLP, en donde los pesos de las representaciones vectoriales están "congeladas" y no son parámetros que el modelo deba optimizar. En consecuencia, estos modelos son más rápidos de entrenar y permiten aplicar arquitecturas especificas que sean diferenciales en cada una de las tareas.

Por el otro lado, las técnicas que emplean **Fine-tunning** tiene la flexibilidad de poder adaptar sus representaciones al permitir que todos los parametros sean optimizados en la tarea en particular. Además, estas arquitecturas permiten resolver multiples problemas de NLP utilizando una mínima cantidad de parametros específicos para la tarea.

## BETO: BERT en español

Al igual que con `word2vec`, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días). Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de textos. Pueden encontrar más información sobre en [el sitio web del autor](https://github.com/dccuchile/beto).

Fue entrenado por la universidad de chile. Lo recomienda Santiago para Español.
Son modelos que necesitan Tarjeta Gráfica.

### Tokenizers

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Recuerden que los tokenizers dependen del modelo con el que estamos trabajando:

In [ ]:
!pip install transformers[torch]
!pip install accelerate>=0.20.1

In [ ]:
#pip install transformers
!pip install transformers

In [ ]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased',
                                                           do_lower_case=True)

*Noten que el tokenizer depende del modelo que estamos utilizando*

## Crando un modelo de clasificación basado en BERT

Trataremos de resolver entonces el mismo problema de clasificación con el que veniamos trabajando: clasificar los tweets dependiendo del sector al que pertenecen.Recordemos que tenemos 7 categorias distintas:

In [ ]:
labels = base['categoria'].unique().tolist()
labels

['No_aplica', 'No_Culpable', 'Culpable']

In [ ]:
base.head()

,num_pol,num_Stro,sinister_detail,categoria,Caracteres
0,5380929,1156984,.,No_aplica,1
1,999999999,1340973,s,No_aplica,1
2,5420691,1146105,-,No_aplica,1
3,7050199,1268001,H,No_aplica,1
4,5538353,1147194,De,No_aplica,2


Necesitaremos contar con el numero de categorias para nuestro clasificador:

In [ ]:
num_labels=len(labels)
num_labels

3

Antes de hacer fine-tunning de nuestro modelo, tenemos que instanciar el modelo sobre el cual queremos aplicar esta técnica. Para ello instanciaremos el modelo base el cual no está entrenado en ninguna tarea en particular. De hecho, si habilitan las alertas en este notebook, verán que cuando se carga el modelo, la libreria HuggingFace les advierte sobre esto:

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased',
                                                      num_labels=num_labels) #Esta es la URL del modelo que queremos utilizar.
#La librería Transformers necesita una vectorización específica al igual que el tokenizer correcto.
#Meterse en la web de Hugging Face para ver varios modelos preentrenados.
#Si necesitamos Regresión, cambiamos a BertForSequenceClassification.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Construiremos nuestro dataset sobre el que queremos entrenar el modelo. Recuerden que ya habíamos separado el set de datos en porciones para entrenar y para testear el modelo.

### Como entrenar modelos con Transformers

La librería transformers puede entrenar modelos tanto utilizando TensorFlow como PyTorch como backend. En nuestro caso utilizaremos PyTorch simplemente porque generaremos código un poco más compacto, pero pueden utilizar el backend con el que más cómodos se sientan:

### Set de datos

Para utilizar el objeto `Trainer` que provee `transformers`, necesitamos crear un objetivo de tipo `Dataset`. `PyTorch` implementa este objeto el modulo `torch.utils.data.Dataset`. Para simplificar la tarea, disponemos de una clase que hace todo el procesamiento de datos y generación de los conjuntos de datos utilizando dicho modulo. Pueden encontrar esta implementación en `Utils.ClassificationDataset`:

In [ ]:
from Utils.TextDataset import ClassificationDataset

In [ ]:
train_dataset = ClassificationDataset(examples=X_train, labels=y_train, tokenizer=tokenizer)
val_dataset = ClassificationDataset(examples=X_test, labels=y_test, tokenizer=tokenizer)

In [ ]:
from Utils.TextDataset import ClassificationDataset

train_dataset = ClassificationDataset(examples=X_train, labels=y_train, tokenizer=tokenizer)
val_dataset = ClassificationDataset(examples=X_test, labels=y_test, tokenizer=tokenizer)

In [ ]:
base.dtypes

num_pol             int64
num_Stro            int64
sinister_detail    object
categoria          object
Caracteres          int64
dtype: object

In [ ]:
#pip install transformers
!pip install transformers

In [ ]:
from transformers import Trainer, TrainingArguments

### Trainer

Especificamos los parametros con los que entrenaremos nuestro modelo:

In [ ]:
!pip install transformers[torch]
!pip install accelerate>=0.20.1

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de trabajo del Trainer
    num_train_epochs=3,              # Numero total de epochs sobre el que entrenaremos
    per_device_train_batch_size=16,  # Tamaño del batch de datos por cada dispositivo de entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch de datos que usaremos para evaluación
    warmup_steps=500,                # Numero de pasos que se usaran para determinar la politica de Learning Rate
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directorio de logs
)

Instanciamos el Trainer

In [ ]:
trainer = Trainer(
    model=model,                         # modelo sobre el que haremos fine tunning
    args=training_args,                  # parametros del entrenamiento
    train_dataset=train_dataset,         # set de datos de entrenamiento
    eval_dataset=val_dataset,             # set de datos de evaluación
)

In [ ]:
history = trainer.train()
#Está aprendiendo a utilizar el texto para tratar de clasificar. BERT solo entiende de lenguaje.
#Esto es mucho más económico que resolver a BERT completo.
#Se puede aumentar la cantidad de epochs.

Step,Training Loss
500,0.556400
1000,0.376800
1500,0.344300
2000,0.322600
2500,0.297600
3000,0.277100
3500,0.255300
4000,0.259900
4500,0.257700
5000,0.241900


Podemos inspeccionar nuestro modelo utilizando la herramienta `TensorBoard`

In [ ]:
#!tensorboard --logdir ./logs --bind_all

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

En Google Colab, podemos ver `TensorBoard` usando el siguiente `Magic`

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir ./logs

Verifiquemos la performance de nuestro modelo

In [ ]:
predictions = trainer.predict(test_dataset=val_dataset).predictions

Para evaluar el modelo, primero deberemos obtener cual es la categoria que obtuvo la mayor probabilidad en la clasificación:

In [ ]:
import numpy as np

predictions = np.argmax(predictions, axis=1) #Prediction tiene el array con todas las probabilidades.
#Argmax nos devuelve la más probable.

Convertimos los IDs de las categorias a los labels correctos

In [ ]:
all_labels = val_dataset.get_labels()
predictions_label = [all_labels[idx] for idx in predictions]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions_label))

              precision    recall  f1-score   support

    Culpable       0.87      0.86      0.87      3932
 No_Culpable       0.91      0.91      0.91      5974
   No_aplica       0.98      1.00      0.99       381

    accuracy                           0.90     10287
   macro avg       0.92      0.93      0.92     10287
weighted avg       0.90      0.90      0.90     10287



### Persistiendo el modelo

Podemos persistir el modelo para utilizar posteriormente de la siguiente forma. Note que deberemos persistir tanto el modelo como el `tokenizer` ya que los dos funcionan de la mano.

In [ ]:
model_name = "culpable_o_no_classifier_bert_20230310"

trainer.save_model(model_name)
tokenizer.save_pretrained(model_name)

('culpable_o_no_classifier_bert_20230310/tokenizer_config.json',
 'culpable_o_no_classifier_bert_20230310/special_tokens_map.json',
 'culpable_o_no_classifier_bert_20230310/vocab.txt',
 'culpable_o_no_classifier_bert_20230310/added_tokens.json',
 'culpable_o_no_classifier_bert_20230310/tokenizer.json')

De las lineas anteriores, vemos que tenemos dos elementos para persistir, el modelo propiamente dicho y su correspondiente tokenizer. De hecho, si exploramos un poco más a detalle los elementos que se guardaron, veremos varios archivos que se generan, incluyendo, por ejemplo, el vocabulario que el tokenizer utilizará.

Una forma conveniente de empaquetar todos estos archivos podría ser comprimirlos en un único archivo para moverlo de un lado al otro de forma sencilla:

In [ ]:
!zip -r culpable_o_no_classifier_bert_20230310.zip culpable_o_no_classifier_bert_20230310

  adding: culpable_o_no_classifier_bert_20230310/ (stored 0%)
  adding: culpable_o_no_classifier_bert_20230310/special_tokens_map.json (deflated 42%)
  adding: culpable_o_no_classifier_bert_20230310/pytorch_model.bin (deflated 7%)
  adding: culpable_o_no_classifier_bert_20230310/config.json (deflated 51%)
  adding: culpable_o_no_classifier_bert_20230310/vocab.txt (deflated 56%)
  adding: culpable_o_no_classifier_bert_20230310/tokenizer.json (deflated 71%)
  adding: culpable_o_no_classifier_bert_20230310/tokenizer_config.json (deflated 44%)
  adding: culpable_o_no_classifier_bert_20230310/training_args.bin (deflated 49%)
